In [2]:
import os
import sys
import copy
import glob
import h5py
import json

import util
import util_bez2018model
import util_tfrecords


In [4]:
"""
Generate config files for bez2018model nervegrams:
word and voice recognition datasets.
"""

BASE_CONFIG = {
    'examples_per_tfrecord_fin': 1000,
    'examples_per_tfrecord_tmp': 20,
    'compression_type': 'GZIP',
    'disp_step': 1,
    'kwargs_ExampleProcessor': {
        'kwargs_nervegram': {
            'nervegram_dur': None,
            'nervegram_dtype': 'float16',
            'nervegram_fs': 10e3,
            'buffer_start_dur': 0.0,
            'buffer_end_dur': 0.0,
            'pin_fs': 100e3,
            'pin_dBSPL_flag': 0,
            'pin_dBSPL': None,
            'species': 2,
            'bandwidth_scale_factor': 1.0,
            'cf_list': None,
            'num_cf': 50,
            'min_cf': 125,
            'max_cf': 8e3,
            'max_spikes_per_train': 1000,
            'num_spike_trains': 1,
            'cohc': 1.0,
            'cihc': 1.0,
            'IhcLowPass_cutoff': 3000,
            'IhcLowPass_order': 7,
            'spont': [70.0, 4.0, 0.1],
            'noiseType': 1,
            'implnt': 0,
            'tabs': 6e-4,
            'trel': 6e-4,
            'synapseMode': 0,
            'random_seed': None,
            'return_vihcs': False,
            'return_meanrates': True,
            'return_spike_times': False,
            'return_spike_tensor_sparse': False,
            'return_spike_tensor_dense': False,
            'nervegram_spike_tensor_fs': 100e3,
            'squeeze_spont_dim': True,
            'squeeze_channel_dim': True,
        },
        'key_signal_fs': '/sources/signal/sr',
        'key_signal': '/stimuli/signal',
        'key_noise': None,
        'key_snr': None,
        'key_dbspl': None,
        'range_snr': None,
        'range_dbspl': [30.0, 90.0],
        'list_keys_to_copy': [
            ["/augment_STRAIGHT_f0_shift", "augment_STRAIGHT_f0_shift"],
            ["/augment_STRAIGHT_whisper", "augment_STRAIGHT_whisper"],
            ["/sources/noise/labels_binary_via_int", "label_audioset_multihot"],
            ["/sources/signal/speaker_int", "label_speaker_int"],
            ["/sources/signal/word_int", "label_word_int"],
            ['/sources/noise/snr', 'snr'],
        ],
        'list_keys_to_ignore': [
            'pin'
        ],
    }
}

list_dir_stim = [
    'stimuli/spkr_word_recognition/optimization/train',
    'stimuli/spkr_word_recognition/optimization/valid',
    'stimuli/spkr_word_recognition/evaluation/hopkins_moore_2009',
    'stimuli/spkr_word_recognition/evaluation/human_experiment_v00_foreground60dbspl',
    'stimuli/spkr_word_recognition/evaluation/human_experiment_v00_inharmonic_foreground60dbspl',
    'stimuli/spkr_word_recognition/evaluation/pitch_altered_v00',
    'stimuli/spkr_word_recognition/evaluation/speech_in_synthetic_textures',
]

for dir_stim in list_dir_stim:
    fn_src = glob.glob(os.path.join(dir_stim, '*.hdf5'))[0]
    CONFIG = copy.deepcopy(BASE_CONFIG)
    print('\n' + fn_src)
    if 'evaluation' in fn_src:
        CONFIG['kwargs_ExampleProcessor']['key_signal'] = 'signal'
        CONFIG['kwargs_ExampleProcessor']['key_signal_fs'] = 'sr'
        CONFIG['kwargs_ExampleProcessor']['range_dbspl'] = None
        CONFIG['kwargs_ExampleProcessor']['list_keys_to_copy'] = []
        with h5py.File(fn_src, 'r') as f_src:
            for k in util.get_hdf5_dataset_key_list(f_src):
                skip_list = [
                    'foreground_wav',
                    'foreground_wav_clip_pos',
                    'background_wav',
                    'background_wav_clip_pos',
                    'signal',
                    'background_signal',
                    'foreground_signal',
                ]
                if k not in skip_list:
                    CONFIG['kwargs_ExampleProcessor']['list_keys_to_copy'].append(k)
                    assert len(f_src[k].shape) == 1

    for itr, IhcLowPass_cutoff in enumerate([3000, 1000, 320, 50]):
        dir_nervegram = os.path.join(dir_stim, 'tfrecords_IHC{:04.0f}'.format(IhcLowPass_cutoff))
        CONFIG['kwargs_ExampleProcessor']['kwargs_nervegram']['IhcLowPass_cutoff'] = IhcLowPass_cutoff
        fn_config = os.path.join(dir_nervegram, 'config.json')
        if not os.path.exists(dir_nervegram):
            os.mkdir(dir_nervegram)
            print(dir_nervegram)
        if itr == 0:
            example_processor = util_bez2018model.ExampleProcessor(
                fn_src,
                idx_start=0,
                idx_end=100,
                random_seed=0,
                **CONFIG['kwargs_ExampleProcessor'])
            example = example_processor.process_example(0)
            feature_description, bytes_description = util_tfrecords.get_description_from_example(example)
            example_processor.close()
        CONFIG['feature_description'] = feature_description
        CONFIG['bytes_description'] = bytes_description
        util_bez2018model.write_config(dir_nervegram, CONFIG)



stimuli/spkr_word_recognition/optimization/train/JSIN_all__run_000_DZKGBOFKNVV2SVR5ZNV7KC5TRMD6P5ZJ.hdf5
###### EXAMPLE STRUCTURE ######
## augment_STRAIGHT_f0_shift float32 () -1.0
## augment_STRAIGHT_whisper float32 () 0.0
## dbspl float64 () 62.92881023563949
## label_audioset_multihot int64 (517,) 4136
## label_speaker_int int64 () 385
## label_word_int int64 () 28
## nervegram_meanrates float16 (50, 20000, 3) 6000000
## signal float32 (40000,) 160000
## signal_fs int64 () 20000
## snr float64 () 2.0
###### EXAMPLE STRUCTURE ######
[WROTE CONFIG] stimuli/spkr_word_recognition/optimization/train/tfrecords_IHC3000/config_feature_description.pkl
[WROTE CONFIG] stimuli/spkr_word_recognition/optimization/train/tfrecords_IHC3000/config_bytes_description.pkl
[WROTE CONFIG] stimuli/spkr_word_recognition/optimization/train/tfrecords_IHC3000/config.json
[WROTE CONFIG] stimuli/spkr_word_recognition/optimization/train/tfrecords_IHC1000/config_feature_description.pkl
[WROTE CONFIG] stimuli/spk

In [3]:
"""
Generate config files for bez2018model nervegrams:
sound localization datasets.
"""

BASE_CONFIG = {
    'examples_per_tfrecord_fin': 1000,
    'examples_per_tfrecord_tmp': 20,
    'compression_type': 'GZIP',
    'disp_step': 1,
    'kwargs_ExampleProcessor': {
        'kwargs_nervegram': {
            'nervegram_dur': 1.3,
            'nervegram_dtype': 'float16',
            'nervegram_fs': 10e3,
            'buffer_start_dur': 0.07,
            'buffer_end_dur': 0.01,
            'pin_fs': 100e3,
            'pin_dBSPL_flag': 0,
            'pin_dBSPL': None,
            'species': 2,
            'bandwidth_scale_factor': 1.0,
            'cf_list': None,
            'num_cf': 50,
            'min_cf': 125,
            'max_cf': 16e3,
            'max_spikes_per_train': 1000,
            'num_spike_trains': 1,
            'cohc': 1.0,
            'cihc': 1.0,
            'IhcLowPass_cutoff': 3000,
            'IhcLowPass_order': 7,
            'spont': [70.0, 4.0, 0.1],
            'noiseType': 0,
            'implnt': 0,
            'tabs': 6e-4,
            'trel': 6e-4,
            'synapseMode': 0,
            'random_seed': None,
            'return_vihcs': False,
            'return_meanrates': True,
            'return_spike_times': False,
            'return_spike_tensor_sparse': False,
            'return_spike_tensor_dense': False,
            'nervegram_spike_tensor_fs': 100e3,
        },
        'key_signal_fs': 'sr',
        'key_signal': 'signal',
        'key_noise': None,
        'key_snr': None,
        'key_dbspl': None,
        'range_snr': None,
        'range_dbspl': None,
        'list_keys_to_copy': [],
        'list_keys_to_ignore': [
            'pin'
        ],
    }
}

list_dir_stim = [
    'stimuli/sound_localization/optimization/train',
    'stimuli/sound_localization/optimization/valid',
    'stimuli/sound_localization/evaluation/bandwidth_dependency',
    'stimuli/sound_localization/evaluation/itd_ild_weighting',
    'stimuli/sound_localization/evaluation/itd_threshold',
    'stimuli/sound_localization/evaluation/minimum_audible_angle_interpolated',
    'stimuli/sound_localization/evaluation/mp_spectral_cues',
    'stimuli/sound_localization/evaluation/new_ears',
    'stimuli/sound_localization/evaluation/precedence_effect_localization',
    'stimuli/sound_localization/evaluation/spectral_smoothing',
    'stimuli/sound_localization/evaluation/speech_in_noise_in_reverb_v04',
    'stimuli/sound_localization/evaluation/v01_eval_mit_bldg46room1004_tenoise',
]

for dir_stim in list_dir_stim:
    fn_src = glob.glob(os.path.join(dir_stim, '*.hdf5'))[0]
    CONFIG = copy.deepcopy(BASE_CONFIG)
    print('\n' + fn_src)
    with h5py.File(fn_src, 'r') as f_src:
        if f_src['signal'].shape[1] > 61740:
            CONFIG['kwargs_ExampleProcessor']['sample_start'] = int(0.28 * 44e3)
            CONFIG['kwargs_ExampleProcessor']['sample_end'] = int((0.28 + 1.40) * 44e3)
            CONFIG['kwargs_ExampleProcessor']['kwargs_nervegram']['nervegram_dur'] = 1.3
        elif f_src['signal'].shape[1] == 44100:
            CONFIG['kwargs_ExampleProcessor']['kwargs_nervegram']['nervegram_dur'] = 1.0
        for k in util.get_hdf5_dataset_key_list(f_src):
            skip_list = [
                'foreground_wav',
                'foreground_wav_clip_pos',
                'background_wav',
                'background_wav_clip_pos',
                'signal',
                'background_signal',
                'foreground_signal',
            ]
            if k not in skip_list:
                CONFIG['kwargs_ExampleProcessor']['list_keys_to_copy'].append(k)
                assert len(f_src[k].shape) == 1

    for itr, IhcLowPass_cutoff in enumerate([3000, 1000, 320, 50]):
        dir_nervegram = os.path.join(dir_stim, 'tfrecords_IHC{:04.0f}'.format(IhcLowPass_cutoff))
        CONFIG['kwargs_ExampleProcessor']['kwargs_nervegram']['IhcLowPass_cutoff'] = IhcLowPass_cutoff
        fn_config = os.path.join(dir_nervegram, 'config.json')
        if not os.path.exists(dir_nervegram):
            os.mkdir(dir_nervegram)
            print(dir_nervegram)
        if itr == 0:
            example_processor = util_bez2018model.ExampleProcessor(
                fn_src,
                idx_start=0,
                idx_end=100,
                random_seed=0,
                **CONFIG['kwargs_ExampleProcessor'])
            example = example_processor.process_example(0)
            feature_description, bytes_description = util_tfrecords.get_description_from_example(example)
            example_processor.close()
        CONFIG['feature_description'] = feature_description
        CONFIG['bytes_description'] = bytes_description
        util_bez2018model.write_config(dir_nervegram, CONFIG)



stimuli/sound_localization/optimization/train/stim_0000000_0009071.hdf5
###### EXAMPLE STRUCTURE ######
## dbspl float32 () 62.92881
## flag_complete int64 () 1
## foreground_azim int64 () 80
## foreground_dist float32 () 1.4
## foreground_elev int64 () 20
## foreground_index_brir int64 () 182
## index int64 () 0
## index_background int64 () 26281
## index_foreground int64 () 4908
## index_room int64 () 1484
## len_list_background_index_brir int64 () 7
## list_background_index_brir int64 (7,) [1017  229   70 1514 1153  510  631]
## n_loc_background int64 () 7
## n_loc_foreground int64 () 1
## nervegram_meanrates float16 (50, 13000, 3, 2) 7800000
## signal float32 (57330, 2) 458640
## signal_fs int64 () 44100
## snr float32 () 2.3158484
## split_int int64 () 0
## sr int64 () 44100
###### EXAMPLE STRUCTURE ######
[WROTE CONFIG] stimuli/sound_localization/optimization/train/tfrecords_IHC3000/config_feature_description.pkl
[WROTE CONFIG] stimuli/sound_localization/optimization/train/tfrec

In [4]:
"""
Generate config files for bez2018model nervegrams with higher sampling rate:
word and voice recognition datasets.
"""

BASE_CONFIG = {
    'examples_per_tfrecord_fin': 1000,
    'examples_per_tfrecord_tmp': 20,
    'compression_type': 'GZIP',
    'disp_step': 1,
    'kwargs_ExampleProcessor': {
        'kwargs_nervegram': {
            'nervegram_dur': None,
            'nervegram_dtype': 'float16',
            'nervegram_fs': 20e3,
            'buffer_start_dur': 0.0,
            'buffer_end_dur': 0.0,
            'pin_fs': 100e3,
            'pin_dBSPL_flag': 0,
            'pin_dBSPL': None,
            'species': 2,
            'bandwidth_scale_factor': 1.0,
            'cf_list': None,
            'num_cf': 50,
            'min_cf': 125,
            'max_cf': 8e3,
            'max_spikes_per_train': 1000,
            'num_spike_trains': 1,
            'cohc': 1.0,
            'cihc': 1.0,
            'IhcLowPass_cutoff': 3000,
            'IhcLowPass_order': 7,
            'spont': [70.0, 4.0, 0.1],
            'noiseType': 1,
            'implnt': 0,
            'tabs': 6e-4,
            'trel': 6e-4,
            'synapseMode': 0,
            'random_seed': None,
            'return_vihcs': False,
            'return_meanrates': True,
            'return_spike_times': False,
            'return_spike_tensor_sparse': False,
            'return_spike_tensor_dense': False,
            'nervegram_spike_tensor_fs': 100e3,
            'squeeze_spont_dim': True,
            'squeeze_channel_dim': True,
        },
        'key_signal_fs': '/sources/signal/sr',
        'key_signal': '/stimuli/signal',
        'key_noise': None,
        'key_snr': None,
        'key_dbspl': None,
        'range_snr': None,
        'range_dbspl': [30.0, 90.0],
        'list_keys_to_copy': [
            ["/augment_STRAIGHT_f0_shift", "augment_STRAIGHT_f0_shift"],
            ["/augment_STRAIGHT_whisper", "augment_STRAIGHT_whisper"],
            ["/sources/noise/labels_binary_via_int", "label_audioset_multihot"],
            ["/sources/signal/speaker_int", "label_speaker_int"],
            ["/sources/signal/word_int", "label_word_int"],
            ['/sources/noise/snr', 'snr'],
        ],
        'list_keys_to_ignore': [
            'pin'
        ],
    }
}

list_dir_stim = [
    'stimuli/spkr_word_recognition/optimization/train',
    'stimuli/spkr_word_recognition/optimization/valid',
    'stimuli/spkr_word_recognition/evaluation/hopkins_moore_2009',
    'stimuli/spkr_word_recognition/evaluation/human_experiment_v00_foreground60dbspl',
    'stimuli/spkr_word_recognition/evaluation/human_experiment_v00_inharmonic_foreground60dbspl',
    'stimuli/spkr_word_recognition/evaluation/pitch_altered_v00',
    'stimuli/spkr_word_recognition/evaluation/speech_in_synthetic_textures',
]

for dir_stim in list_dir_stim:
    fn_src = glob.glob(os.path.join(dir_stim, '*.hdf5'))[0]
    CONFIG = copy.deepcopy(BASE_CONFIG)
    print('\n' + fn_src)
    if 'evaluation' in fn_src:
        CONFIG['kwargs_ExampleProcessor']['key_signal'] = 'signal'
        CONFIG['kwargs_ExampleProcessor']['key_signal_fs'] = 'sr'
        CONFIG['kwargs_ExampleProcessor']['range_dbspl'] = None
        CONFIG['kwargs_ExampleProcessor']['list_keys_to_copy'] = []
        with h5py.File(fn_src, 'r') as f_src:
            for k in util.get_hdf5_dataset_key_list(f_src):
                skip_list = [
                    'foreground_wav',
                    'foreground_wav_clip_pos',
                    'background_wav',
                    'background_wav_clip_pos',
                    'signal',
                    'background_signal',
                    'foreground_signal',
                ]
                if k not in skip_list:
                    CONFIG['kwargs_ExampleProcessor']['list_keys_to_copy'].append(k)
                    assert len(f_src[k].shape) == 1

    for itr, IhcLowPass_cutoff in enumerate([3000]):
        dir_nervegram = os.path.join(dir_stim, 'tfrecords_IHC{:04.0f}_sr20000'.format(IhcLowPass_cutoff))
        CONFIG['kwargs_ExampleProcessor']['kwargs_nervegram']['IhcLowPass_cutoff'] = IhcLowPass_cutoff
        fn_config = os.path.join(dir_nervegram, 'config.json')
        if not os.path.exists(dir_nervegram):
            os.mkdir(dir_nervegram)
            print(dir_nervegram)
        if itr == 0:
            example_processor = util_bez2018model.ExampleProcessor(
                fn_src,
                idx_start=0,
                idx_end=100,
                random_seed=0,
                **CONFIG['kwargs_ExampleProcessor'])
            example = example_processor.process_example(0)
            feature_description, bytes_description = util_tfrecords.get_description_from_example(example)
            example_processor.close()
        CONFIG['feature_description'] = feature_description
        CONFIG['bytes_description'] = bytes_description
        util_bez2018model.write_config(dir_nervegram, CONFIG)



stimuli/spkr_word_recognition/optimization/train/JSIN_all__run_000_DZKGBOFKNVV2SVR5ZNV7KC5TRMD6P5ZJ.hdf5
###### EXAMPLE STRUCTURE ######
## augment_STRAIGHT_f0_shift float32 () -1.0
## augment_STRAIGHT_whisper float32 () 0.0
## dbspl float64 () 62.92881023563949
## label_audioset_multihot int64 (517,) 4136
## label_speaker_int int64 () 385
## label_word_int int64 () 28
## nervegram_meanrates float16 (50, 40000, 3) 12000000
## signal float32 (40000,) 160000
## signal_fs int64 () 20000
## snr float64 () 2.0
###### EXAMPLE STRUCTURE ######
[WROTE CONFIG] stimuli/spkr_word_recognition/optimization/train/tfrecords_IHC3000_sr20000/config_feature_description.pkl
[WROTE CONFIG] stimuli/spkr_word_recognition/optimization/train/tfrecords_IHC3000_sr20000/config_bytes_description.pkl
[WROTE CONFIG] stimuli/spkr_word_recognition/optimization/train/tfrecords_IHC3000_sr20000/config.json

stimuli/spkr_word_recognition/optimization/valid/JSIN_all__run_000_RQTTZB4C3TJJVLJUWDV72TYMC7S4MNHH.hdf5
###### E

In [7]:
"""
Generate config files for bez2018model nervegrams with higher sampling rate:
sound localization datasets.
"""

BASE_CONFIG = {
    'examples_per_tfrecord_fin': 1000,
    'examples_per_tfrecord_tmp': 20,
    'compression_type': 'GZIP',
    'disp_step': 1,
    'kwargs_ExampleProcessor': {
        'kwargs_nervegram': {
            'nervegram_dur': 1.3,
            'nervegram_dtype': 'float16',
            'nervegram_fs': 20e3,
            'buffer_start_dur': 0.07,
            'buffer_end_dur': 0.01,
            'pin_fs': 100e3,
            'pin_dBSPL_flag': 0,
            'pin_dBSPL': None,
            'species': 2,
            'bandwidth_scale_factor': 1.0,
            'cf_list': None,
            'num_cf': 50,
            'min_cf': 125,
            'max_cf': 16e3,
            'max_spikes_per_train': 1000,
            'num_spike_trains': 1,
            'cohc': 1.0,
            'cihc': 1.0,
            'IhcLowPass_cutoff': 3000,
            'IhcLowPass_order': 7,
            'spont': [70.0, 4.0, 0.1],
            'noiseType': 0,
            'implnt': 0,
            'tabs': 6e-4,
            'trel': 6e-4,
            'synapseMode': 0,
            'random_seed': None,
            'return_vihcs': False,
            'return_meanrates': True,
            'return_spike_times': False,
            'return_spike_tensor_sparse': False,
            'return_spike_tensor_dense': False,
            'nervegram_spike_tensor_fs': 100e3,
        },
        'key_signal_fs': 'sr',
        'key_signal': 'signal',
        'key_noise': None,
        'key_snr': None,
        'key_dbspl': None,
        'range_snr': None,
        'range_dbspl': None,
        'list_keys_to_copy': [],
        'list_keys_to_ignore': [
            'pin'
        ],
    }
}

list_dir_stim = [
    'stimuli/sound_localization/optimization/train',
    'stimuli/sound_localization/optimization/valid',
    'stimuli/sound_localization/evaluation/bandwidth_dependency',
    'stimuli/sound_localization/evaluation/itd_ild_weighting',
    'stimuli/sound_localization/evaluation/itd_threshold',
    'stimuli/sound_localization/evaluation/minimum_audible_angle_interpolated',
    'stimuli/sound_localization/evaluation/mp_spectral_cues',
    'stimuli/sound_localization/evaluation/new_ears',
    'stimuli/sound_localization/evaluation/precedence_effect_localization',
    'stimuli/sound_localization/evaluation/spectral_smoothing',
    'stimuli/sound_localization/evaluation/speech_in_noise_in_reverb_v04',
    'stimuli/sound_localization/evaluation/v01_eval_mit_bldg46room1004_tenoise',
]

for dir_stim in list_dir_stim:
    fn_src = glob.glob(os.path.join(dir_stim, '*.hdf5'))[0]
    CONFIG = copy.deepcopy(BASE_CONFIG)
    print('\n' + fn_src)
    with h5py.File(fn_src, 'r') as f_src:
        if f_src['signal'].shape[1] > 61740:
            CONFIG['kwargs_ExampleProcessor']['sample_start'] = int(0.28 * 44e3)
            CONFIG['kwargs_ExampleProcessor']['sample_end'] = int((0.28 + 1.40) * 44e3)
            CONFIG['kwargs_ExampleProcessor']['kwargs_nervegram']['nervegram_dur'] = 1.3
        elif f_src['signal'].shape[1] == 44100:
            CONFIG['kwargs_ExampleProcessor']['kwargs_nervegram']['nervegram_dur'] = 1.0
        for k in util.get_hdf5_dataset_key_list(f_src):
            skip_list = [
                'foreground_wav',
                'foreground_wav_clip_pos',
                'background_wav',
                'background_wav_clip_pos',
                'signal',
                'background_signal',
                'foreground_signal',
            ]
            if k not in skip_list:
                CONFIG['kwargs_ExampleProcessor']['list_keys_to_copy'].append(k)
                assert len(f_src[k].shape) == 1

    for itr, IhcLowPass_cutoff in enumerate([3000]):
        dir_nervegram = os.path.join(dir_stim, 'tfrecords_IHC{:04.0f}_sr20000'.format(IhcLowPass_cutoff))
        CONFIG['kwargs_ExampleProcessor']['kwargs_nervegram']['IhcLowPass_cutoff'] = IhcLowPass_cutoff
        fn_config = os.path.join(dir_nervegram, 'config.json')
        if not os.path.exists(dir_nervegram):
            os.mkdir(dir_nervegram)
            print(dir_nervegram)
        if itr == 0:
            example_processor = util_bez2018model.ExampleProcessor(
                fn_src,
                idx_start=0,
                idx_end=100,
                random_seed=0,
                **CONFIG['kwargs_ExampleProcessor'])
            example = example_processor.process_example(0)
            feature_description, bytes_description = util_tfrecords.get_description_from_example(example)
            example_processor.close()
        CONFIG['feature_description'] = feature_description
        CONFIG['bytes_description'] = bytes_description
        util_bez2018model.write_config(dir_nervegram, CONFIG)



stimuli/sound_localization/optimization/train/stim_0000000_0009071.hdf5
stimuli/sound_localization/optimization/train/tfrecords_IHC3000_sr20000
###### EXAMPLE STRUCTURE ######
## dbspl float32 () 62.92881
## flag_complete int64 () 1
## foreground_azim int64 () 80
## foreground_dist float32 () 1.4
## foreground_elev int64 () 20
## foreground_index_brir int64 () 182
## index int64 () 0
## index_background int64 () 26281
## index_foreground int64 () 4908
## index_room int64 () 1484
## len_list_background_index_brir int64 () 7
## list_background_index_brir int64 (7,) [1017  229   70 1514 1153  510  631]
## n_loc_background int64 () 7
## n_loc_foreground int64 () 1
## nervegram_meanrates float16 (50, 26000, 3, 2) 15600000
## signal float32 (57330, 2) 458640
## signal_fs int64 () 44100
## snr float32 () 2.3158484
## split_int int64 () 0
## sr int64 () 44100
###### EXAMPLE STRUCTURE ######
[WROTE CONFIG] stimuli/sound_localization/optimization/train/tfrecords_IHC3000_sr20000/config_feature_d

In [3]:
"""
Modify network architectures to operate on 2x higher sampling rate nervegrams
"""
list_dir_model_src = glob.glob("models/sound_localization/IHC3000*/arch*")

for dir_model_src in list_dir_model_src:
    dir_model_dst = dir_model_src.replace("IHC3000", "sr20000_IHC3000")
    assert not dir_model_dst == dir_model_src
    if not os.path.exists(dir_model_dst):
        os.mkdir(dir_model_dst)
    with open(os.path.join(dir_model_src, "config.json"), "r") as f:
        config_model = json.load(f)
    with open(os.path.join(dir_model_src, "arch.json"), "r") as f:
        architecture = json.load(f)

    if "sound_localization" in dir_model_dst:
        config_model["kwargs_dataset_from_tfrecords"]["features_to_exclude"].append("list_background_index_brir")
        config_model["kwargs_cochlea"]["sr_cochlea"] = 20e3
        config_model["kwargs_cochlea"]["sr_input"] = 20e3
        config_model["kwargs_cochlea"]["sr_output"] = 20e3
        architecture[0]["args"]["kernel_size"][1] *= 2
        architecture[1]["args"]["pool_size"][1] *= 2
    else:
        raise NotImplementedError("Only sound_localization model modification is implemented")

    with open(os.path.join(dir_model_dst, "config.json"), "w") as f:
        json.dump(config_model, f, indent=4, sort_keys=True)
    with open(os.path.join(dir_model_dst, "arch.json"), "w") as f:
        json.dump(architecture, f, indent=4, sort_keys=True)

    print(f'"{dir_model_dst}"')


"models/sound_localization/sr20000_IHC3000/arch01"
"models/sound_localization/sr20000_IHC3000/arch02"
"models/sound_localization/sr20000_IHC3000/arch03"
"models/sound_localization/sr20000_IHC3000/arch04"
"models/sound_localization/sr20000_IHC3000/arch05"
"models/sound_localization/sr20000_IHC3000/arch06"
"models/sound_localization/sr20000_IHC3000/arch07"
"models/sound_localization/sr20000_IHC3000/arch08"
"models/sound_localization/sr20000_IHC3000/arch09"
"models/sound_localization/sr20000_IHC3000/arch10"
"models/sound_localization/sr20000_IHC3000_delayed_integration/arch01"
"models/sound_localization/sr20000_IHC3000_delayed_integration/arch02"
"models/sound_localization/sr20000_IHC3000_delayed_integration/arch03"
"models/sound_localization/sr20000_IHC3000_delayed_integration/arch04"
"models/sound_localization/sr20000_IHC3000_delayed_integration/arch05"
"models/sound_localization/sr20000_IHC3000_delayed_integration/arch06"
"models/sound_localization/sr20000_IHC3000_delayed_integration/a